# CNNによる画像分類

画像分類とは、画像の種類や状態を定量的に解析して特定のクラスに分類するタスクである。  
ここでは、細胞画像を生細胞と死細胞に分類するタスクを学習するためのCNNを構築し、訓練および評価までの一連の作業を行う。

## データセット

`dataset_cls/` にHeLa細胞の位相差顕微鏡画像300枚 (生細胞150枚, 死細胞150枚) が格納されている。  
画像サイズはすべて 128x128 pixel で、グレースケール画像となっている。  

まず最初に画像を読み込み表示してみる。

In [ ]:
import skimage.io as io

%matplotlib inline
img_live = io.imread('../datasets/dataset_cls/live/001.tif')
io.imshow(img_live)

In [ ]:
img_dead = io.imread('../datasets/dataset_cls/dead/001.tif')
io.imshow(img_dead)

## 学習に用いるデータセットの準備

データセットは、次の３つのカテゴリに分割する必要がある。

* **train**: 訓練に用いるデータセット
* **validation**: モデルの構造を決定するパラメータ (ハイパーパラメータ) を決めるために用いるデータセット
* **test**: 検証に用いるデータセット

それぞれどの画像がどのデータセットにカテゴライズされているか、`datasets/dataset_cls/split_list` にあるテキストファイル (`train.txt`, `validation.txt`, `test.txt`) にリストしてある。
これらをカテゴライズしておく理由については、のちに説明する。

また、画像分類の学習に用いるデータは、画像とラベル (今回は**死細胞のラベルを0**、**生細胞のラベルを1**とする)　をペアで用意する必要がある。

あるカテゴリのデータセットから、画像とラベルをペアで用意するためのクラスを次のように作成する。

In [ ]:
import os
import numpy as np
import skimage.io as io
import chainer

class PreprocessedDataset(chainer.dataset.DatasetMixin):
    def __init__(
            self,
            root_path,
            split_list
    ):
        self.root_path = root_path
        with open(split_list) as f:
            self.split_list = [line.rstrip() for line in f]
        self.dtype = np.float32

    def __len__(self):
        return len(self.split_list)

    # 画像を返す
    def _get_image(self, i):
        image = io.imread(os.path.join(self.root_path, self.split_list[i]))
        image = self._min_max_normalize_one_image(image)
        return np.expand_dims(image.astype(self.dtype), axis=0)

    # 画像を[0, 1]に正規化
    def _min_max_normalize_one_image(self, image):
        max_int = image.max()
        min_int = image.min()
        out = (image.astype(np.float32) - min_int) / (max_int - min_int)
        return out
    
    # ラベル (生:1, 死: 0) を返す
    def _get_label(self, i):
        label = 0 if 'dead' in self.split_list[i] else 1
        return label

    # 画像とそれに紐づいたラベル (生:1, 死: 0) を返す
    def get_example(self, i):
        x, y = self._get_image(i), self._get_label(i)
        return x, y

このクラスでは、引数に渡した `split_list` (`train.txt`, `validation.txt`, `test.txt`のどれか) に従って、画像とラベルのペアを出力するための `get_example()` 関数が用意してある。  
実際に、`get_example()` を次のように動かしてみる。

In [ ]:
root_path = '../datasets/dataset_cls'
split_list = '../datasets/dataset_cls/split_list/train.txt'

dataset = PreprocessedDataset(root_path, split_list)

%matplotlib inline
# train.txtの0番目の画像とラベルを取得
img, label = dataset.get_example(0)
plt.figure()
plt.imshow(img[0], cmap='gray')
plt.title('label: {}'.format(label))

# train.txtの80番目の画像とラベルを取得
img, label = dataset.get_example(80)
plt.figure()
plt.imshow(img[0], cmap='gray')
plt.title('label: {}'.format(label))

画像とラベルが正しい組み合わせで取得できていることが確認できた。

## CNNモデルの定義

画像分類を行うCNNは、畳み込み層と全結合層に分かれている。  
畳み込み層は `L.Convolution2D(in_ch, out_ch, ksize, stride, pad)` ,
全結合層は  `L.Linear(in_ch, out_ch)` のように記述する。  
ここで指定する値は、モデルの構造を決定するパラメータ、すなわちハイパーパラメータと呼ばれる。

```
    in_ch:   入力チャネル数
    out_ch:  出力チャネル数
    ksize:   畳み込みカーネルサイズ
    stride:  カーネルの移動サイズ
    pad:     パディングサイズ
```

また、順伝搬されたデータを正規化するための層を正規化層と呼び、
`L.BatchNormalization(in_ch)` のように記述する。

参考: https://docs.chainer.org/en/stable/reference/links.html

ここまで説明した層 (畳み込み層,  全結合層, 正規化層) はすべてパラメータを持っている。  
一方で、パラメータを持たない、つまり学習によって更新はされない層や関数が存在する。

CNNにおいてよく用いられる次元削減を行うための層としてプーリング層がある。
プーリング層は `F.max_pooling_2d(input, ksize, stride)` のように記述する。
また、代表的な活性化関数としてReLU (Rectified Linear Unit) があり、`F.relu(input)` と記述することができる。

参考: https://docs.chainer.org/en/stable/reference/functions.html

In [ ]:
import chainer
import chainer.functions as F
import chainer.links as L

class ClassificationModel(chainer.Chain):
    def __init__(self, n_class=2):
        super(ClassificationModel, self).__init__()
        with self.init_scope():
            # ネットワークの部品 (層) を用意
            self.conv1 = L.Convolution2D(1, 32, 5, 1, 2)
            self.bn1 = L.BatchNormalization(32)
            self.conv2 = L.Convolution2D(32, 64, 5, 1, 2)
            self.bn2 = L.BatchNormalization(64)
            self.conv3 = L.Convolution2D(64, 128, 3, 1, 1)
            self.bn3 = L.BatchNormalization(128)
            self.conv4 = L.Convolution2D(128, 256, 3, 1, 1)
            self.bn4 = L.BatchNormalization(256)
            self.fc5 = L.Linear(16384, 1024)
            self.fc6 = L.Linear(1024, n_class)
            
    def __call__(self, x):
        # 各層を接続
        h = F.relu(self.conv1(x))
        h = F.max_pooling_2d(self.bn1(h), 2, 2)
        h = F.relu(self.conv2(h))
        h = F.max_pooling_2d(self.bn2(h), 2, 2)
        h = F.relu(self.conv3(h))
        h = F.max_pooling_2d(self.bn3(h), 2, 2)
        h = F.relu(self.conv4(h))
        h = F.max_pooling_2d(self.bn4(h), 2, 2)
        h = F.dropout(F.relu(self.fc5(h)))
        return self.fc6(h)

## 出力関数

ここまででCNNモデルに用いられる関数と伝搬の仕方を定義したが、最後の出力される層には別の関数を用いる。  
この出力関数を用いることで、問題に応じた値の範囲になるように調節することができる。  
一般的に、分類問題では出力関数に**Softmax**関数が用いられる。

Softmax関数は、$d$次元のベクトル${\bf y} \in \mathbb{R}^d$が与えられたとき、その各次元の値の合計が1になるように正規化することができる。
すなわち、確率分布のような出力となる。
${\bf y}$の$i$番目の次元を$y_i$と書くと，Softmax関数は以下のように定義される。

$$
{\rm Softmax}({\bf y})_i = \frac{\exp(y_i)}{\sum_{j=1}^d \exp(y_j)}
$$

このように表現されるSoftmax関数を出力関数として用いる場合、`F.softmax` と記述する。

参考: https://docs.chainer.org/en/stable/reference/generated/chainer.functions.softmax.html

## CNNモデルによる推論

ここまでで、学習に用いるデータセットとCNNモデルを用意することができた。  
まずはこれらを用いて、実際に画像分類の推論ができるのか試してみる。

In [ ]:
import chainer.functions as F
import chainer.links as L

root_path = '../datasets/dataset_cls'
split_list = '../datasets/dataset_cls/split_list/test.txt'

# データセットを用意
test_dataset = PreprocessedDataset(root_path, split_list)

# 未学習のモデルを作成
model = L.Classifier(ClassificationModel(n_class=2))

print('=====================')
for i in range(5):
    with chainer.using_config('train', False):
        # テスト画像とラベルを1セット取得
        img, label = test_dataset.get_example(i)
        # テスト画像の生死を推論
        pred = model.predictor(np.expand_dims(img, axis=0))
        # 出力関数により生・死の確率を求める
        pred = F.softmax(pred)

    print('test {}'.format(i + 1))
    print('  pred: {}'.format(np.argmax(pred.data)))     # 確率が高い方のラベルを表示
    print('  label: {}'.format(label))                                   # 正解のラベルを表示
    print('=====================')

このように、画像をCNNモデルに入力して、出力された結果から画像分類の推論を確認することができる。  
ただ、このCNNモデルは学習が行われていない状態なので、画像分類の結果もランダムに出力した結果とさほど変わらないことがわかる。  
では次に、すでに用意してある学習済みCNNモデルを使って推論を行なってみる。

In [ ]:
import chainer.functions as F
import chainer.links as L
from chainer import serializers

root_path = '../datasets/dataset_cls'
split_list = '../datasets/dataset_cls/split_list/test.txt'
model_path = '../models/inference_model'


# データセットを用意
test_dataset = PreprocessedDataset(root_path, split_list)

# 未学習のモデルを作成
model = L.Classifier(ClassificationModel(n_class=2))

# 学習済みモデルの読み込み
serializers.load_npz(model_path, model)


print('=====================')
for i in range(5):
    with chainer.using_config('train', False):
        # テスト画像とラベルを1セット取得
        img, label = test_dataset.get_example(i)
        # テスト画像の生死を推論
        pred = model.predictor(np.expand_dims(img, axis=0))
        # 出力関数により生・死の確率を求める
        pred = F.softmax(pred)

    print('test {}'.format(i + 1))
    print('  pred: {}'.format(np.argmax(pred.data)))     # 確率が高い方のラベルを表示
    print('  label: {}'.format(label))                                   # 正解のラベルを表示
    print('=====================')

学習済みモデルを用いた結果、推論結果と正解ラベルがすべて一致していることがわかる。  
では次からは、学習を行うための準備を行なっていく。

## 目的関数

ここでは、上で定義したCNNモデルの目的関数 (誤差関数, 損失関数とも呼ばれる) を定義する。  
目的関数はタスクごとに設定し、分類問題の場合は **Softmax Cross Entropy** 関数を用いるのが一般的である。  
Softmax Cross Entropy関数は、Softmax関数とCross Entropy関数を組み合わせた関数である。

Softmax関数は上述したため、ここではCross Entropy (交差エントロピー) 関数について説明する。
$N$クラスの分類問題の場合、ある入力$x$が与えられたとき、ニューラルネットワークの出力層に$N$個のノードがあり、
それぞれが$n$番目のクラスに属する確率$y_n$を表しているとする。
ここで、$x$が所属するクラスについての正解が、${\bf t} = \begin{bmatrix} t_1 & t_2 & \dots & t_N \end{bmatrix}^T$というベクトルで与えられているとする。
ただし、このベクトルは $t_n (n = 1, 2, \dots, N)$ のいずれか1つだけが1であり、
それ以外は0であるようなベクトル (1-hot ベクトル) であるとする。
そして、この1つだけ値が1となっている要素は、その要素のインデックスに対応したクラスが正解であることを意味する。
このとき、Cross Entropy関数は、以下のように定義される。

$$
{\rm Cross\ Entropy}({\bf y}, {\bf t}) = - \frac{1}{N} \sum_{n=1}^{N}t_{n}\log({\rm Softmax}({\bf y})_{n})
$$

このように表現されるSoftmax Cross Entropy関数を目的関数として用いる場合、`F.softmax_cross_entropy` と記述する。

参考: https://docs.chainer.org/en/stable/reference/generated/chainer.functions.softmax_cross_entropy.html

## CNNモデルの目的関数を定義

ここでは、CNNモデルの出力から目的関数を用いて誤差 (loss) を計算するためのクラスを作成する。

In [ ]:
import chainer
import chainer.functions as F
from chainer import reporter, Variable

class Classifier(chainer.Chain):
    def __init__(self, predictor):
        super(Classifier, self).__init__()
        with self.init_scope():
            self.predictor = predictor

    def __call__(self, x, t):
        y = self.predictor(x)
        # 目的関数を設定
        loss = F.softmax_cross_entropy(y, t)
        # 誤差を記録
        reporter.report({'loss': loss}, self)
        return loss

## 最適化手法

ニューラルネットワークにおける学習は、上述した目的関数を最小化するために最適化を行う必要がある。
この最適化手法は勾配法がベースとなっており、学習時に用いる最適化手法はこれまでに数多く提案されている (参考: https://qiita.com/tokkuman/items/1944c00415d129ca0ee9 )。

Chainerにも多くの最適化手法が実装されているが、今回はその中でも最も用いられている**Adam**という最適化手法を採用する。  
Adamを用いる場合には、`optimizers.Adam` と記述する。

参考: https://docs.chainer.org/en/stable/reference/optimizers.html

## CNNモデルの学習

CNNモデルに目的関数を定義して、最適化手法も決定したため、実際に学習を行なってみる。

In [ ]:
import chainer
from chainer import cuda
import chainer.functions as F
from chainer import optimizers, iterators, training

def create_trainer(root_path, train_path, val_path, batchsize, epoch, out='result_cls', device=-1):

    # train データセットを用意
    train_dataset = PreprocessedDataset(root_path, train_path)
    # validation データセットを用意
    val_dataset = PreprocessedDataset(root_path, val_path)
    
    # 未学習のモデルを作成
    model = ClassificationModel(n_class=2)
    train_model = Classifier(model)

    # 最適化手法を選択 (Adam)
    optimizer = optimizers.Adam()
    optimizer.setup(train_model)

    # train データセットを学習器に渡す形式に変換
    train_iter = iterators.MultiprocessIterator(train_dataset, batchsize)
    # validation データセットを学習器に渡す形式に変換
    val_iter = iterators.MultiprocessIterator(val_dataset, batchsize, repeat=False, shuffle=False)

    # 誤差から学習器の重みを更新するアップデーターを作成
    updater = training.StandardUpdater(train_iter, optimizer, device=device)

    # 今回実行したいタスクを設定
    trainer = training.trainer.Trainer(updater, epoch, out=out)

    # 学習中に何を記録するかを設定 (エポック, 訓練誤差, 検証誤差, 実行時間)
    logging_attributes = ['epoch', 'main/loss', 'val/main/loss', 'elapsed_time']
    # 上記 logging_attributes をログとして記録
    trainer.extend(training.extensions.LogReport(logging_attributes))
    # 上記 logging_attributes を画面に表示   
    trainer.extend(training.extensions.PrintReport(logging_attributes))
    # 訓練誤差と検証誤差をプロット   
    trainer.extend(training.extensions.PlotReport(['main/loss', 'val/main/loss'], 'epoch', file_name='loss.png'))
    # 検証用データの評価の設定
    trainer.extend(training.extensions.Evaluator(val_iter, train_model, device=device), name='val')
    # 検証誤差が最小時のモデルを保存
    trainer.extend(training.extensions.snapshot_object(train_model, filename='best_loss_model'), trigger=training.triggers.MinValueTrigger('val/main/loss', trigger=(1, 'epoch')))
    return trainer


os.makedirs('results', exist_ok=True)
root_path = '../datasets/dataset_cls'
train_path = '../datasets/dataset_cls/split_list/train.txt'
val_path = '../datasets/dataset_cls/split_list/validation.txt'
batchsize = 2
epoch = (10, 'epoch')

trainer = create_trainer(root_path, train_path, val_path, batchsize, epoch, out='results/result_cls_1', device=-1)

# 訓練開始
trainer.run()

学習が進むにつれて訓練データの誤差 (`main/loss`) が低下していることがわかる。
つまり、学習が正常に進んでいることがわかる。
このように、lossを継時的に観察することで、学習の進行度を確認することができる。
また、検証データの誤差 (`val/main/loss`) がもっとも低いとき、学習が収束したと判定することができる。
一方で、学習の評価指標が誤差のみだと、実際に画像分類がうまく行われているのか確認することができない。
そこで、学習時に観察する指標を新たに導入することを考える。

## 画像分類精度の評価指標

一般的に、分類問題の精度は**Accuracy**という指標で評価される。Accuracyは全画像のうち何枚の画像を正しく分類できたかを割合で表される指標である。すなわち、Accuracyが高いほど分類精度も高いということになる。 
正解クラスが$i$である画像をCNNモデルがクラス$j$に分類した数を$N_{ij}$とすると、クラス数が$k$のとき、Accuracyは以下のようになる。

$$
{\rm Accuracy} = \frac{\sum_{i=1}^k N_{ii}}{\sum_{i=1}^k \sum_{j=1}^k N_{ij}}
$$

Accuracyを用いる場合、`F.accuracy` と記述する。

参考: https://docs.chainer.org/en/stable/reference/generated/chainer.functions.accuracy.html

In [ ]:
import chainer
import chainer.functions as F
from chainer import reporter, Variable

class Classifier(chainer.Chain):
    def __init__(self, predictor):
        super(Classifier, self).__init__()
        with self.init_scope():
            self.predictor = predictor

    def __call__(self, x, t):
        y = self.predictor(x)
        # 目的関数を設定
        loss = F.softmax_cross_entropy(y, t)
        with chainer.no_backprop_mode():
            # 分類精度を計算
            accuracy = F.accuracy(y, t)
        # 誤差, 分類精度を記録
        reporter.report({'loss': loss, 'accuracy': accuracy}, self)
        return loss

## Accuracyを指標とした学習

先ほどまではlossのみで学習の進行度と収束判定を行なっていたが、今度は導入したAccuracyを用いて同様に学習してみる。

In [ ]:
import chainer
from chainer import cuda
import chainer.functions as F
from chainer import optimizers, iterators, training

def create_trainer(root_path, train_path, val_path, batchsize, epoch, out='result_cls', device=-1):

    train_dataset = PreprocessedDataset(root_path, train_path)
    val_dataset = PreprocessedDataset(root_path, val_path)
    
    model = ClassificationModel(n_class=2)
    train_model = Classifier(model)

    optimizer = optimizers.Adam()
    optimizer.setup(train_model)

    train_iter = iterators.MultiprocessIterator(train_dataset, batchsize)
    val_iter = iterators.MultiprocessIterator(val_dataset, batchsize, repeat=False, shuffle=False)

    updater = training.StandardUpdater(train_iter, optimizer, device=device)

    trainer = training.trainer.Trainer(updater, epoch, out=out)

    # 学習中に分類精度 (accuracy) も記録するように設定
    logging_attributes = ['epoch', 'main/loss', 'main/accuracy', 'val/main/loss', 'val/main/accuracy', 'elapsed_time']
    trainer.extend(training.extensions.LogReport(logging_attributes))
    trainer.extend(training.extensions.PrintReport(logging_attributes))
    trainer.extend(training.extensions.PlotReport(['main/loss', 'val/main/loss'], 'epoch', file_name='loss.png'))
    # 訓練精度と検証精度をプロット 
    trainer.extend(training.extensions.PlotReport(['main/accuracy', 'val/main/accuracy'], 'epoch', file_name='accuracy.png'))
    trainer.extend(training.extensions.Evaluator(val_iter, train_model, device=device), name='val')
    # 検証精度が最高時のモデルを保存
    trainer.extend(training.extensions.snapshot_object(train_model, filename='best_acc_model'), trigger=training.triggers.MaxValueTrigger('val/main/accuracy', trigger=(1, 'epoch')))
    return trainer


root_path = '../datasets/dataset_cls'
train_path = '../datasets/dataset_cls/split_list/train.txt'
val_path = '../datasets/dataset_cls/split_list/test.txt'
batchsize = 2
epoch = (10, 'epoch')

trainer = create_trainer(root_path, train_path, val_path, batchsize, epoch, out='results/result_cls_2', device=-1)
trainer.run()

## テストデータを用いた画像分類の精度

In [ ]:
import numpy as np
import chainer
from chainer import cuda
import chainer.functions as F
from chainer import serializers

def test(root_path, test_path, model_path, out='results/result_cls_test', device=-1):

    # データセットを用意
    test_dataset = PreprocessedDataset(root_path, test_path)
    
    # 未学習のモデルを作成
    model = ClassificationModel(n_class=2)
    train_model = Classifier(model)
    
    # 学習済みモデルの読み込み
    serializers.load_npz(model_path, train_model)
    
    true_conut = 0
    for i in range(test_dataset.__len__()):
        with chainer.using_config('train', False):
            # テスト画像とラベルを1セット取得
            x, t = test_dataset.get_example(i)
            # テスト画像の生死を推論
            y = train_model.predictor(np.expand_dims(x, axis=0))
        # 推論した分類結果が正解ラベルと一致していたら true_count　に1を加算
        if np.argmax(y.data) == t:
            true_conut += 1
    # accuracy を算出
    accuracy = true_conut / test_dataset.__len__()
    os.makedirs(out, exist_ok=True)
    with open(os.path.join(out, 'result.txt'), 'w') as f:
        f.write('accuracy: {}\n'.format(accuracy))
    print('accuracy: {}'.format(accuracy))
    
    
root_path = '../datasets/dataset_cls'
test_path = '../datasets/dataset_cls/split_list/test.txt'

# 検証誤差が最小の学習済みモデルでテスト
model_path = 'results/result_cls_1/best_loss_model'
test(root_path, test_path, model_path, 'results/result_cls_1_test', device=-1)

# 検証精度が最高の学習済みモデルでテスト
model_path = 'results/result_cls_2/best_acc_model'
test(root_path, test_path, model_path, 'results/result_cls_2_test', device=-1)

## 5分割交差検証 (5-fold cross validation)

交差検証 (cross validation) とは、学習の妥当性を検証・確認するための手法である。
K-分割交差検証 (K-fold cross validation) では、データセットをK個に分割する。
次に、分割した1つのデータセットを検証データとし、残る  K - 1個を訓練データとする。
交差検証は、K 個に分割されたデータセットをそれぞれ検証データとして K 回検証を行う。
このようにして得られた K 回の結果を平均することで、データセット全体に対してうまく学習できているか評価を行うことができる。

In [ ]:
root_path = '../datasets/dataset_cls'
batchsize = 2
epoch = (10, 'epoch')

## fold 1

In [ ]:
train_path_fold1 = '../datasets/dataset_cls/split_list/fold1/train.txt'
val_path_fold1 = '../datasets/dataset_cls/split_list/fold1/validation.txt'

trainer_fold1 = create_trainer(root_path, train_path_fold1, val_path_fold1, batchsize, epoch=epoch, out='results/result_cls_fold1', device=-1)
trainer_fold1.run()

## fold 2

In [ ]:
train_path_fold2 = '../datasets/dataset_cls/split_list/fold2/train.txt'
val_path_fold2 = '../datasets/dataset_cls/split_list/fold2/validation.txt'

trainer_fold2 = create_trainer(root_path, train_path_fold2, val_path_fold2, batchsize, epoch=epoch, out='results/result_cls_fold2', device=-1)
trainer_fold2.run()

## fold 3

In [ ]:
train_path_fold3 = '../datasets/dataset_cls/split_list/fold3/train.txt'
val_path_fold3 = '../datasets/dataset_cls/split_list/fold3/validation.txt'

trainer_fold3 = create_trainer(root_path, train_path_fold3, val_path_fold3, batchsize, epoch=epoch, out='results/result_cls_fold3', device=-1)
trainer_fold3.run()

## fold 4

In [ ]:
train_path_fold4 = '../datasets/dataset_cls/split_list/fold4/train.txt'
val_path_fold4 = '../datasets/dataset_cls/split_list/fold4/validation.txt'

trainer_fold4 = create_trainer(root_path, train_path_fold4, val_path_fold4, batchsize, epoch=epoch, out='results/result_cls_fold4', device=-1)
trainer_fold4.run()

## fold 5

In [ ]:
train_path_fold5 = '../datasets/dataset_cls/split_list/fold5/train.txt'
val_path_fold5 = '../datasets/dataset_cls/split_list/fold5/validation.txt'

trainer_fold5 = create_trainer(root_path, train_path_fold5, val_path_fold5, batchsize, epoch=epoch, out='results/result_cls_fold5', device=-1)
trainer_fold5.run()

## Cross Validationの評価

In [ ]:
import os
import glob
import json
import matplotlib.pylab as plt

res_cv_path = glob.glob('results/result_cls_fold*')
res_log = []
for cv in res_cv_path:
    with open(os.path.join(cv, 'log'), 'r') as f:
        res_log.append(json.load(f))


%matplotlib inline
    
plt.figure(figsize=(18, 10))
for k in range(len(res_log)):
    train, val, epoch = [], [], []
    for n in range(len(res_log[k])):
        train.append(res_log[k][n]['main/accuracy'])
        val.append(res_log[k][n]['val/main/accuracy'])
        epoch.append(res_log[k][n]['epoch'])
    plt.subplot(2, 3, k+1)
    plt.plot(epoch, train, label='Train', color='blue')
    plt.plot(epoch, val, label='Validation', color='orange')
    plt.scatter(np.argmax(val)+1, np.max(val), marker='o', color='orange')
    plt.xticks(epoch)
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('fold{}: max accuracy={}'.format(k+1, np.max(val)))
    plt.legend()

# テストデータによる検証

ここまでで train, validation　データを用いて学習を行ったので、学習に関与させていない test データを用いて得られた学習済みモデルの精度を検証する。
一般的には、交差検証においてもっとも優れた精度を示したモデルを用いて検証を行う。

## テスト関数の作成

In [ ]:
import numpy as np
import chainer
from chainer import cuda
import chainer.functions as F
from chainer import serializers

def test(root_path, test_path, model_path, out='results/result_cls_test', device=-1):
    test_dataset = PreprocessedDataset(root_path, test_path)
    
    model = ClassificationModel(n_class=2)
    train_model = Classifier(model)
    serializers.load_npz(model_path, train_model)
    
    tp, tn, fp, fn = 0, 0, 0, 0
    for i in range(test_dataset.__len__()):
        with chainer.using_config('train', False):
            x, t = test_dataset.get_example(i)
            y = train_model.predictor(np.expand_dims(x, axis=0))
        if np.argmax(y.data) == t:
            if t == 1:
                tp += 1
            else:
                tn += 1
        else:
            if t == 1:
                fn += 1
            else:
                fp += 1
            
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    fscore = 2 * precision * recall / (precision + recall)
    
    os.makedirs(out, exist_ok=True)
    with open(os.path.join(out, 'result.txt'), 'w') as f:
        f.write('accuracy: {}\n'.format(accuracy))
        f.write('precision: {}\n'.format(precision))
        f.write('recall: {}\n'.format(recall))
        f.write('f-score: {}\n'.format(fscore))
    print('accuracy: {}'.format(accuracy))
    print('precision: {}'.format(precision))
    print('recall: {}'.format(recall))
    print('f-score: {}'.format(fscore))

## テスト

In [ ]:
root_path = '../datasets/dataset_cls'
test_path = '../datasets/dataset_cls/split_list/test.txt'
model_path = 'results/result_cls_fold2/best_acc_model'

test(root_path, test_path, model_path, 'results/result_cls_test', device=-1)